# Active work in LIFE Target Database

# starcat 4 stars that would be good to be in there

In [4]:
import numpy as np
starname='TRAPPIST-1'
# find star in db
table=database_tables['ident']
main_id=table['main_id'][np.where(table['id']==starname)][0]
o_idref=table['object_idref'][np.where(table['id']==starname)][0]
print(table[np.where(table['id']==starname)])
objects=database_tables['objects']
print(objects['type'][np.where(objects['main_id']==main_id)])
#find stars properties
star_basic=database_tables['star_basic']
cols=['main_id','class_lum','class_temp','class_temp_nr']
print(star_basic[cols][np.where(star_basic['main_id']==main_id)])

#find parent and sibling
bhlink=database_tables['best_h_link']
print(bhlink[np.where(bhlink['child_object_idref']==o_idref)])
parent_o_idref=bhlink['parent_object_idref'][np.where(bhlink['child_object_idref']==o_idref)][0]
print(bhlink[np.where(bhlink['parent_object_idref']==parent_o_idref)])

#sibling properties
print(star_basic[cols][np.where(star_basic['main_id']=="WDS J10114+4927B")])

#wait, eta cas A is seen as system but does have stellar parameters...
#why is not having a parent a reason for exclusion?

    id      main_id   id_source_idref object_idref
---------- ---------- --------------- ------------
TRAPPIST-1 TRAPPIST-1              92         5040
TRAPPIST-1 TRAPPIST-1              93         5040
type
----
  st
 main_id   class_lum class_temp class_temp_nr
---------- --------- ---------- -------------
TRAPPIST-1         V          M           7.5
parent_main_id child_main_id ... child_object_idref parent_object_idref
                             ...                                       
-------------- ------------- ... ------------------ -------------------


IndexError: index 0 is out of bounds for axis 0 with size 0

- Alp Cen A -> * alf Cen A -> higher order mult
- Alp Cen B -> * alf Cen B -> higher order mult
- Proxima Cen -> * alf Cen C -> higher order mult
- Teegardens Star -> NAME Teegarden\'s Star -> no sep
- Tau Cet -> * tau Cet -> sibling (from WDS so no stellar properties) doesn't fulfill criteria
- Eps Eri -> * eps Eri -> why is here 0.0 in sep??
- Eps Ind -> * eps Ind -> sibling (from WDS so no stellar properties) doesn't fulfill criteria
- HD 88230 -> HD  88230 -> sibling (from WDS so no stellar properties) doesn't fulfill criteria
- omi02 Eri -> * omi02 Eri -> multiple parents in WDS
- sig Dra -> * sig Dra -> multiple parents in WDS
- Gliese 570 -> HD 131977 -> multiple parents in WDS
- eta Cas A -> * eta Cas A -> multiple parents in WDS
- eta Cas B -> * eta Cas B -> multiple parents in WDS
- TRAPPIST-1


In [3]:
from utils.catalog_comparison import detail_criteria
l_golden_targets=['* alf Cen A','* alf Cen B','* alf Cen C','NAME Teegarden\'s Star','* tau Cet','* eps Eri','* eps Ind','HD  88230','* omi02 Eri','* sig Dra','HD 131977','* eta Cas A','* eta Cas B','TRAPPIST-1']
detail_criteria(database_tables,l_golden_targets)
#to do, save star names without NAME in front.

tbd: file starcat4 analysis where this code is used to give specific reason why a given object was not included into the cat4
Of the 14 objects given:
Some are not in cat 4 because they are either:
system_without_child ['HD  88230', '* eta Cas A', '* eta Cas B'] 3
star_without_parent ['TRAPPIST-1'] 1
not found [] 

Some were not included because they are systems and instead one should look at their child objects:
children ["NAME Teegarden's Star b", "NAME Teegarden's Star c", "NAME Teegarden's Star d"] 

some are non trivial binaries:
multiple parents ['* eps Eri', '* omi02 Eri', '* sig Dra', 'HD 131977'] 4
higher_order_multiple ['* alf Cen A', '* alf Cen B', '* alf Cen C'] 3 

And the reminder have conpanions that don t fit the spectral type requirements
trivial binary ['* tau Cet', '* eps Ind'] 2


In [ ]:
"""
LIFE-StarCat4 creation

This script allows the user to create the LIFE-StarCat version 4. 

This script requires that numpy, pyvo and astropy be installed within 
the Python environment you are running this script in.
"""

import numpy as np # Used for arrays
import pyvo as vo # Used for catalog query
import astropy as ap # Used for votables
import importlib # Used by developer to reload external functions
import math
import sys  


# Self created modules
#sys.path.append('../../implementation/life/data_generation/life_td_data_generation') #wait, in that case saving and loading would be at wrong place
from utils.io import save, load
from utils.catalog_comparison import object_contained

def query(link,adql_query,catalogs=[]):
    """
    Performs a TAP query on the service given in the link parameter.
    
    If a list of tables is given in the catalogs parameter,
    those are uploaded to the service.
    
    :param link: Service access URL.
    :type link:
    :param adql_query: Query of the external database service in ADQL.
    :type adql_query:
    :param catalogs: Tables to be uploaded to the service.
    :type catalogs: list of astropy tables
    :returns: Table containing the result of the query.
    :rtype: Astropy table
    """
    
    # Defining the VO service using the given link
    service = vo.dal.TAPService(link)
    
    if catalogs==[]: # without upload tables
        result=service.run_async(adql_query.format(**locals()), maxrec=160000)
    
    else: # with upload tables
        tables={}
        for i in range(len(catalogs)):
            tables.update({f"t{i+1}":catalogs[i]})
        result = service.run_async(adql_query,uploads=tables,timeout=None,
                                   maxrec=160000)
    cat=result.to_table()
    return cat

def crit_sep(eps,mu,a_bin):
    """
    Computes critical semimajor-axis for planet orbit stability.

    For binary system as described in Holman and Wiegert 1999.

    :param eps: Binary orbit excentricity.
    :type eps:
    :param mu: mass fraction with mu=m_s/(m_p+m_s), with m_s the mass 
        of the star considered as perturbing binary companion and m_p 
        the mass of the star the planet is orbiting.
    :type mu:
    :param a_bin: semimajor-axis of the binary stars.
    :type a_bin:
    :returns: Critical separation beyond which a planet on a S-type 
        orbit (circumstellar) and on a P-type orbit (circumbinary) is 
        not stable any more.
    :rtype: 
    """
    a_crit_s=(0.464-0.38*mu-0.631*eps+0.586*mu*eps+0.15*eps**2\
              -0.198*mu*eps**2)*a_bin
    a_crit_p=(1.6+5.1*eps-2.22*eps**2+4.12*mu-4.27*eps*mu-5.09*mu**2\
              +4.61*eps**2*mu**2)*a_bin
    return a_crit_s,a_crit_p

def ecliptic(ang,ra,dec):
    """
    Computes if position is within angle from the ecliptic.

    :param ang: Angle in degrees.
    :type ang:
    :param ra: Right ascention in degrees.
    :type ra: np.array
    :param dec: Array of declination in degrees.
    :type dec: np.array
    :returns: Flags.
    :rtype: np.array
    """
    ecliptic=(23.4)*np.sin(2*np.pi*ra/360)
    flag=['True' if dec[j] > -ang+ecliptic[j] and dec[j] < ang+ecliptic[j] \
            else 'False' for j in range(len(ra))]
    return flag

def StarCat_creation(distance_cut,stars=[],details=False,querying=True):
    """
    LIFE-StarCat4 creation
    
    :param distance_cut:
    :type distance_cut:
    :param
    :type
    :param
    :type
    :param
    :type
    :returns:
    :rtype:
    """
    
    # version 3 parameters were: ra, dec, plx, distance, name, sptype, 
    # coo_gal_l, coo_gal_b, Teff, R, M, sep_phys, binary_flag, mag_i, 
    # mag_j
    adql_query="""
    SELECT o.main_id, sb.coo_ra, sb.coo_dec, sb.plx_value, sb.dist_st_value,
        sb.sptype_string, sb.coo_gal_l, sb.coo_gal_b, sb.teff_st_value,
        sb.mass_st_value, sb.radius_st_value, sb.binary_flag, sb.mag_i_value,
        sb.mag_j_value,  sb.class_lum, sb.class_temp,
        o_parent.main_id AS parent_main_id, sb_parent.sep_ang_value
    FROM life_td.star_basic AS sb
    JOIN life_td.object AS o ON sb.object_idref=o.object_id
    LEFT JOIN life_td.h_link AS h ON o.object_id=h.child_object_idref
    LEFT JOIN life_td.object AS o_parent ON
        h.parent_object_idref=o_parent.object_id
    LEFT JOIN life_td.star_basic AS sb_parent ON
        o_parent.object_id=sb_parent.object_idref
    WHERE o.type = 'st' AND sb.dist_st_value < """+str(distance_cut)
    # we are only interested in object type stars, up to a distance cut 
    # and well defined luminocity class (to sort out objects not around 
    # main sequence)
    service='http://dc.zah.uni-heidelberg.de/tap'
    print(f'Getting stars with distance smaller than {plx_cut}...')
    
    if querying==True:
        print('Querying life_td...')
        cat=query(service,adql_query)
        save([cat],['StarCat4_raw'],location='data/')
    else:
        print('Loading StarCat4_raw...')
        [cat]=load(['StarCat4_raw'],location='data/')
    
    if stars!=[]:
        stars=np.array(stars)
        stars=object_contained(stars,cat['main_id'],details)
        
    print('Removing objects whose temperature class is not in OBAFGKM...')
    ms_tempclass=np.array(['O','B','A','F','G','K','M'])
    cat_ms_tempclass=catalog[np.where(np.in1d(catalog['class_temp'],ms_tempclass))]
    
    if len(stars)>0:
        stars=object_contained(stars,cat_ms_tempclass['main_id'],details)
    
    print('Removing objects that are not main sequence in luminocity class...')
    ms_lumclass=np.array(['V'])
    cat_ms_lumclass=cat_ms_tempclass[np.where(np.in1d(cat_ms_tempclass['class_lum'],ms_lumclass))]
    cat_ms_lumclass.remove_rows(cat_ms_lumclass['mass_st_value'].mask.nonzero()[0])
    
    #separating sample
    singles=cat_ms_lumclass[np.where(cat_ms_lumclass['binary_flag']=='False')]
    multiples=cat_ms_lumclass[np.where(cat_ms_lumclass['binary_flag']=='True')]

    print('Removing higher order multiples:')
    print('... where parent object is a child object as well.')
    #query h_link
    adql_query2="""
    SELECT o.main_id as child_main_id,o.object_id
    FROM life_td.object AS o
    JOIN life_td.h_link AS h on o.object_id=h.child_object_idref
    """
    if querying==True:
        print('Querying life_td for h_link...')
        h_link=query(service,adql_query2)
        save([cat],['h_link'],location='data/')
    else:
        print('Loading h_link...')
        [h_link]=load(['best_h_link'],location='data/')
    #remove those where parent_main_id in h_link as child_main_id
    higher_order_multiples=np.in1d(multiples['parent_main_id'],\
        h_link['child_main_id'])
    multiples.remove_rows(higher_order_multiples)
    
    if len(stars)>0:
        stars=object_contained(stars,
                ap.table.vstack([singles,multiples])['main_id'],details)
    
    print('... where multiple parents or more than two components given.')
    multi_parent=[]
    grouped=multiples.group_by('main_id')
    ind=grouped.groups.indices
    for i in range(len(ind)-1):
        if ind[i+1]-ind[i]!=1:
            multi_parent.append(grouped['main_id'][ind[i]])

    single_parent_multiples=grouped[np.where(np.invert(np.in1d(grouped['main_id'],multi_parent)))]

    if len(stars)>0:
        stars=object_contained(stars,
                ap.table.vstack([singles,single_parent_multiples])['main_id'],details)
        
    #critical sep:
    print('Removing those binaries where no separation value given...')
    sep_multiples=single_parent_multiples[np.where(single_parent_multiples['sep_ang_value'].mask==False)].copy()
    
    if len(stars)>0:
        stars=object_contained(stars,
                ap.table.vstack([singles,multiples])['main_id'],details)
    
    print('Transforming from angular separation into physical one...') 
    #transform into physical separations the remaining ones
    sep_multiples['sep_phys_value']=sep_multiples['sep_ang_value']#just initiating new column with same properties
    sep_multiples['sep_phys_value'].unit=ap.units.AU
    for i in range(len(sep_multiples)):
        sep_multiples['sep_phys_value'][i]=np.round(
                sep_multiples['sep_ang_value'][i]*sep_multiples['dist_st_value'][i],1)    
    
    print('Keeping only binaries where both components fulfill requirements.') 
    # meaning having the parameters we need for critical sep 
    # computation (main sequence & sep given)

    grouped_multiples=sep_multiples.group_by('parent_main_id')
    ind=grouped_multiples.groups.indices

    result=grouped_multiples[:0].copy()

    for i in range(len(ind)-1):
        l=ind[i+1]-ind[i]
        if l==2:
            result.add_row(grouped_multiples[ind[i]])
            result.add_row(grouped_multiples[ind[i]+1])
    
    if len(stars)>0:
        stars=object_contained(stars,
                ap.table.vstack([singles,result])['main_id'],details)
    
    print('Keeping only objects where <10 AU planet orbits are stable...')
    
    result['a_crit_s']=result['sep_phys_value']#initializing column a... like sep...

    for i in range(len(result)):
        m_p=result['mass_st_value'][i]
        if i % 2 == 0:
            m_s=result['mass_st_value'][i+1]
        else:
            m_s=result['mass_st_value'][i-1]
        mu=m_s/(m_p+m_s)
        result['a_crit_s'][i]=crit_sep(0,mu,result['sep_phys_value'][i])[0]
        #assumed circular orbit and sep_phys = a_bin

    final=result[:0].copy()
    #wait, didn't I already define this? -> was before removing some
    ind=result.group_by('parent_main_id').groups.indices
    a_max=10.

    for i in range(len(ind)-1):
        if a_max < min(result['a_crit_s'][ind[i]],result['a_crit_s'][ind[i]+1]):
            final.add_row(result[ind[i]])
            final.add_row(result[ind[i]+1])
    
    if len(stars)>0:
        stars=object_contained(stars,
                ap.table.vstack([singles,final])['main_id'],details)
        
    #does not look right. a_crit is 8 but stayed in sample??

    #what about the inner limit of 0.5??
    
    StarCat4=ap.table.vstack([singles,final])

    print('Adding architecture parameter...')
    # flag any object whose declination is contained within the region 
    # between -(23.4+45)*sin(RA) and +(23.4+45)*sin(RA) with the 
    # object's RA in degrees.
    StarCat4['ecliptic_pm45deg']=ecliptic(45,StarCat4['coo_ra'],\
        StarCat4['coo_dec'])

    save([StarCat4],['StarCat4'],location='data/')
    
    if len(stars)>0:
        print('All criteria were met by:',len(stars))
        if details:
            print(stars)
    
    return StarCat4
    

In [30]:



#'* 212 Pup A' kicked out in non trivial binaries in starcat but in this code as trivial binary with companion not matching sptype
#['* zet01 Ret', '*  50 Per', 'HD  50281', 'HD  65907', '* eps Ind', 'V* V2215 Oph'] other way round

# database creation

In [2]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

#provider_tables_dict, database_tables= ltd.partial_create(30,create=['life'])

provider_tables_dict, database_tables= ltd.create_life_td(20)

#issue: does not wokr with loading as then id_source_idref already exists -> put in a workaround with a warning. though now did not happen...

#issue: create not working right now because exomercat server not available -> put in a workaround
#issue: main_id missing in star_basic -> suddenly working??

Building life_td data with distance cut of 20 pc
Trying to create SIMBAD tables from 2025-05-07...


 sorting object types...
removing 17  objects that had object types: ['BD?|NIR', 'BD?|MIR', 'BD?|MIR|NIR']
example object of them: CWISEP J104756.81+545741.6


Trying to create Grant Kennedy Disks tables from 2024-02-09...
Trying to create WDS tables from 2025-05-07...
Creating  WDS  tables ...
 querying VizieR for WDS...
 performing distance cut...


Trying to create Exo-MerCat tables from 2025-05-07...


Trying to create LIFE tables from 2025-05-07...
Trying to create Gaia tables from 2025-05-07...


Building sources table ...
Building objects table ...


Building provider table ...
Building ident table ...


Building best_h_link table ...
Building star_basic table ...
Building planet_basic table ...
Building disk_basic table ...
Building mes_mass_pl table ...
Building mes_teff_st table ...
Building mes_radius_st table ...
Building mes_mass_st table ...
Building mes_binary table ...
Building mes_sep_ang table ...
Building h_link table ...


Unifying null values...
Saving data...


# load and analyze database

In [2]:
import importlib #reloading external functions after modification

#self created modules
import life_td as ltd
importlib.reload(ltd)#reload module after changing it

provider_tables_dict, database_tables= ltd.load_life_td()


Loading life_td generated data
Loading sim data
Loading sdb data
Loading wds data
Loading exo data
Loading life data
Loading gaia data


Finding issue that causes M0-M5 stars to have too high temperatures (4250 instead of 3000)

In [42]:
print(database_tables)#from create is a dict, from load is a list

{'sources': <Table length=102>
                       ref                            provider_name    source_id
                      object                              object         int64  
-------------------------------------------------- ------------------- ---------
                               1967AJ.....72.1334C              SIMBAD         1
                               1980AJ.....85..454H              SIMBAD         2
                               1982A&A...108..406K              SIMBAD         3
                               1982ApJ...253..237C              SIMBAD         4
                               1982MSS...C03....0H              SIMBAD         5
                               1983SAAOC...7..106W              SIMBAD         6
                               1984ApJ...276..602G              SIMBAD         7
                               1985ApJS...59..197B              SIMBAD         8
                               1988AJ.....95.1841G              SIMBAD        

In [6]:
import numpy as np
#9=mesteff
#5=starbasic
#print(database_tables[5].colnames)
cols=['main_id','sptype_string','teff_st_qual','teff_st_value','teff_st_source_idref']
mstars=database_tables['star_basic'][cols][np.where(database_tables['star_basic']['class_temp']=='M')]
mstars_tempgiven=mstars[np.where(mstars['teff_st_value']!=1e+20)]
print(mstars_tempgiven[np.where(mstars_tempgiven['teff_st_value']>4000)])

  main_id   sptype_string teff_st_qual   teff_st_value   teff_st_source_idref
                                               K                             
----------- ------------- ------------ ----------------- --------------------
BD+00  4050         M0.0V            C            4119.0                  875
BD+24  2733           M3V            C            4110.0                  875
BD-13  1434           M0V            B   4269.0712890625                  874
CD-26 11751            M1            C            4008.0                  875
CD-34  1169             M            D            4250.0                  875
CD-36  9107            M0            B 4095.856689453125                  874
CD-49 14193           M0V            C            4059.0                  875
  HD 166348           M0V            B    4235.400390625                  874
 HD 230017A           M0V            C            4084.0                  875
  HD 330856            M1            C            4105.0        

In [11]:
print(database_tables['sources'][np.where(database_tables['sources']['source_id']==874)])

            ref              provider_name source_id
---------------------------- ------------- ---------
2022arXiv220800211G GSP-Phot          Gaia       874


In [9]:
import numpy as np
#so lets find out if in gaia archive this is also the case
#first find gaia id
cols=['teff_st_qual','teff_st_value','teff_st_source_idref']

print(database_tables['mes_teff_st'][cols][np.where(database_tables['mes_teff_st']['main_id']=='HD 166348')])
#Gaia DR3 4848140361962951552

#on archive only gspphot showable and it is empty

teff_st_qual teff_st_value  teff_st_source_idref
                   K                            
------------ -------------- --------------------
           C         3850.0                  465
           B         4108.0                  875
           B 4235.400390625                  874


In [60]:
from sdata import empty_dict
from provider.utils import fetch_main_id, IdentifierCreator, fill_sources_table, create_sources_table, ids_from_ident, replace_value, create_provider_table, query
from pyvo.dal import TAPService

adql_query="""
    SELECT top 10 s.source_id ,p.mass_flame, p.radius_flame,
        p.teff_gspphot, p.teff_gspspec, m.nss_solution_type, p.age_flame
    FROM gaiadr3.gaia_source as s
        JOIN gaiadr3.astrophysical_parameters as p ON s.source_id=p.source_id
            LEFT JOIN gaiadr3.nss_two_body_orbit as m ON s.source_id=m.source_id
            WHERE s.source_id = 4848140361962951552"""
gaia = empty_dict.copy()
gaia['provider'] = create_provider_table('Gaia',
                                  "https://gea.esac.esa.int/tap-server/tap",
                                  '2016A&A...595A...1G')
try: 
    gaia_helptab=query(gaia['provider']['provider_url'][0],adql_query) 

except:
    #because of bug in gaia server where async not working currently
    service = TAPService(gaia['provider']['provider_url'][0])
    result=service.run_sync(adql_query.format(**locals()), maxrec=160000)
    gaia_helptab=result.to_table()


Trying to create Gaia tables from 2025-02-10...


In [62]:
print(gaia_helptab['source_id','teff_gspspec'])

     source_id      teff_gspspec
                         K      
------------------- ------------
4848140361962951552       4250.0


In [ ]:
#so yes that is how it is in gaia. now find out if this parameter is reliable and if not lower its quality

In [ ]:
import utils.analysis as lta

lta.sanity_tests(database_tables,5)